# Предобрботка третьей бд

In [31]:
import pandas as pd
import numpy as np
import chemparse
from pymatgen.core.composition import Composition

data3 = pd.read_excel('data/Database_3.xlsx')

In [32]:
data3 = data3.drop(columns=['No'])

### Переименовываем признаки

In [33]:
data3.rename(
    columns = {
        'Year': 'year',
        'Material': 'material_type',
        'Type': 'type',
        'Shape': 'shape',
        'Coat/Functional Group': 'coat',
        'Synthesis_Method': 'synthesis_method',
        'Surface_Charge': 'surface_charge_type',
        'Diameter (nm)': 'diameter',
        'Size_in_Water (nm)': 'size_in_water',
        'Size_in_Medium (nm)': 'size_in_medium',
        'Zeta_in_Water (mV)': 'surface_charge',
        'Zeta_in_Medium (mV)': 'zeta_in_medium', 
        'Cell_Type': 'cell_line', 
        'No_of_Cells (cells/well)': 'no_of_cells',
        'Human_Animal': 'human/animal', 
        'Cell_Source': 'animal', 
        'Cell_Tissue': 'cell_organ', 
        'Cell_Morphology': 'cell_morphology',
        'Cell_Age': 'cell_age', 
        'Cell Line_Primary Cell': 'cell_line_bin', 
        'Time (hr)': 'time',
        'Concentration (ug/ml)': 'concentration', 
        'Test': 'test', 
        'Test_Indicator': 'test_indicator', 
        'Aspect_Ratio': 'aspect_ratio',
        'Cell_Viability (%)': 'viability', 
        'PDI': 'pdi', 
        'Article_ID': 'particle_id', 
        'DOI': 'reference_doi'
        }, 
        inplace = True)

### Переименуем соединения

In [34]:
mt_dict = {
    'IronOxide': 'Fe2O3',
    'Dendrmer': 'Dendrimer', 
    'HAP': 'Ca10(PO4)6(OH)2',
    'IronOide': 'Fe2O3',
    'Ay': 'Fe', 
    'Ce O2': 'CeO2'
    }
data3['material_type'].replace(mt_dict, inplace = True)

In [35]:
data3.replace({'None': np.nan}, inplace = True)

### Работа с полем *surface_charge*

In [36]:
data3.loc[320, 'surface_charge'] = 150

In [37]:
data3['surface_charge'] = pd.to_numeric(data3['surface_charge'])

### Добавим молярную массу

In [38]:
without_formula = [
    'QD', 'Dendrimer', 'PLGA', 'Polystyrene',
    'Alginate', 'PTFE-PMMA', 'MWCNT', 
    '(C8H8)n', 'Chitosan','SLN',
    'EudragitRL', 'Carbon', 'C60',
    'C70', 'SWCNT', 'Diamond', 'Graphite'
    ]

In [39]:
data3['molecular_weight'] = data3['material_type'].apply(lambda x: Composition(chemparse.parse_formula(x)).weight if x not in without_formula else np.nan)

In [40]:
c_lst = ['MWCNT', 'Carbon', 'SWCNT', 'MWCNT', 'Diamond', 'Graphite', 'QDs']

data3.loc[data3.material_type.isin(c_lst), 'molecular_weight'] = 12.01
data3.loc[data3.material_type == 'Dendrimer', 'molecular_weight'] = 516.7
data3.loc[data3.material_type == 'PLGA', 'molecular_weight'] = 148.11
data3.loc[data3.material_type == 'Alginate', 'molecular_weight'] = 216.12
data3.loc[data3.material_type == 'PTFE-PMMA', 'molecular_weight'] = 100.12
data3.loc[data3.material_type == 'Dendrimer', 'molecular_weight'] = 516.7
data3.loc[data3.material_type == 'Polystyrene', 'molecular_weight'] = 104.1
data3.loc[data3.material_type == 'Chitosan', 'molecular_weight'] = 501.5
data3.loc[data3.material_type == 'EudragitRL', 'molecular_weight'] = 231.29
data3.loc[data3.material_type == 'Diamond', 'molecular_weight'] = 12.01
data3.loc[data3.material_type == 'C70', 'molecular_weight'] = 840.7
data3.loc[data3.material_type == 'C60', 'molecular_weight'] = 720.66
data3.loc[data3.material_type == 'Graphite', 'molecular_weight'] = 12.01
data3.loc[data3.material_type == 'SLN', 'molecular_weight'] = data3.molecular_weight.mean()

### Добавим электроотрицательность

In [41]:
data3['electronegativity'] = data3['material_type'].apply(lambda x: Composition(chemparse.parse_formula(x)).average_electroneg if x not in without_formula else np.nan)

### Сохраняем

In [42]:
data3.to_excel('data/data3_prep.xlsx', index=False)